## Load Library

In [1]:
import math
import time
import itertools

import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline

## Read File

In [2]:
inFile = open("tsp.txt", "r")

## Convert File to Pandas

In [3]:
arr_list = []

inFile.seek(0)
for line in inFile.readlines():
    line = line[:-1]
    arr_list.append(line.split(" "))
    
df = pd.DataFrame(arr_list, columns = ["Name", "X", "Y"])

# Convert Value From String to int
df["X"] = df["X"].astype(int)
df["Y"] = df["Y"].astype(int)

df

,Name,X,Y
0,Depot,265,143
1,Parcel1,253,278
2,Point2,439,148
3,Parcel3,458,304
4,Point4,609,230
5,Parcel5,597,101


In [4]:
def distance(p1, p2):
    return math.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)

In [5]:
class Point:
    def __init__(self, name, x, y):
        self.name = name
        self.x = x
        self.y = y
        
    def toString(self):
        return f"Name: {self.name}, X Pos: {self.x}, Y Pos: {self.y}"

In [6]:
def create_point(ind):
    row = df.iloc[ind, :]
    return Point(row.Name, row.X, row.Y)

In [7]:
def add_edge_to_graph(G, e1, e2, w):
    G.add_edge(e1, e2, weight=w)

## Create Point Array

In [8]:
pt_arr = []

inFile.seek(0)
for line in inFile.readlines():
    line = line[:-1]
    tmp_arr = line.split(" ")
    
    pt_name = tmp_arr[0]
    x_pos = int(tmp_arr[1])
    y_pos = int(tmp_arr[2])
    
    pt = Point(pt_name, x_pos, y_pos)
    
    pt_arr.append(pt)
    
name_arr = list(map(lambda x : x.name, pt_arr))
name_arr

['Depot', 'Parcel1', 'Point2', 'Parcel3', 'Point4', 'Parcel5']

## Create Edge Array

In [9]:
adj_mat = []

name_arr = list(map(lambda x : x.name, pt_arr))

dist_val = 0

for pt in pt_arr:
    
    name = pt.name
    
    tmp_arr = []
    
    for pt2 in pt_arr:
        dist_val = round(distance(pt, pt2), 3)
        if pt.name == pt2.name or ("Depot" in pt.name and "Point" in pt2.name) or ("Point" in pt.name and "Point" in pt2.name):
            dist_val = float('inf')
        tmp_arr.append(dist_val)
    
    adj_mat.append(tmp_arr)
    
adj_mat

[[inf, 135.532, inf, 251.336, inf, 334.646],
 [135.532, inf, 226.927, 206.642, 359.221, 386.866],
 [174.072, 226.927, inf, 157.153, inf, 164.842],
 [251.336, 206.642, 157.153, inf, 168.158, 246.028],
 [354.831, 359.221, inf, 168.158, inf, 129.557],
 [334.646, 386.866, 164.842, 246.028, 129.557, inf]]

In [10]:
def gen_adj_mat(adj_mat, column_arr):
    tmp_adj_mat = []
    for row_ind in range(len(adj_mat)):
        if row_ind in column_arr:
            tmp_list = []
            for col_ind in column_arr:
                tmp_list.append(adj_mat[row_ind][col_ind])
            tmp_adj_mat.append(tmp_list)
            
    return tmp_adj_mat

In [11]:
gen_adj_mat(adj_mat, [0, 2, 3, 4, 5])

[[inf, inf, 251.336, inf, 334.646],
 [174.072, inf, 157.153, inf, 164.842],
 [251.336, 157.153, inf, 168.158, 246.028],
 [354.831, inf, 168.158, inf, 129.557],
 [334.646, 164.842, 246.028, 129.557, inf]]

## Create Path

In [12]:
class Path:
    def __init__(self, path, cost):
        self.path = path
        self.cost = cost
        
    def __lt__(self, other):
        return self.cost <= other.cost
        
    def __str__(self):
        return f"Shortest Path:{self.path}\nMinimum Cost: {self.cost}"
    
    def __eq__(self, other):
        return self.path == other.path and self.cost == other.cost
    
    def getNodePath(self, name_arr):
        tmp_arr = list(map(lambda x : name_arr[x], self.path))
        return "->".join(tmp_arr)

## Dynamic TSP

In [13]:
ans = gen_adj_mat(adj_mat, [0, 2, 3, 4, 5])
ans

[[inf, inf, 251.336, inf, 334.646],
 [174.072, inf, 157.153, inf, 164.842],
 [251.336, 157.153, inf, 168.158, 246.028],
 [354.831, inf, 168.158, inf, 129.557],
 [334.646, 164.842, 246.028, 129.557, inf]]

In [14]:
def check_tuple(arr, condition_arr):
    flag = True
    for point in condition_arr:
        if point not in arr:
            flag = False
    return flag and len(arr) == (len(condition_arr) * 2 + 1)
    

In [15]:
def retrace_optimal_path(dist_arr, memo, n, condition_arr):
    
    for k, v in memo.items():
        print(k, v)
    
    full_path_memo = dict((k, v) for k, v in memo.items() if check_tuple(k[0], condition_arr))
    
    # Update Cost with Cost Back to Origin
    for key, val in full_path_memo.items():
        visited_tuple, last_point = key
        prev_dist, prev_last_point = val
        memo[key] = (prev_dist + dist_arr[last_point][0], prev_last_point)
        full_path_memo[key] = (prev_dist + dist_arr[last_point][0], prev_last_point)
    path_key = min(full_path_memo.keys(), key=lambda x: full_path_memo[x][0])
    
    print(path_key)
    
    last_point = path_key[1]
    optimal_cost, next_to_last_point = memo[path_key]
    
    points_to_retrace = tuple(sorted(set(path_key[0]).difference({last_point})))
    
    optimal_path = [0] * (2 * len(condition_arr) + 2)
    
    ind = 2 * len(condition_arr)
    while next_to_last_point is not None:
        optimal_path[ind] = last_point
        last_point = next_to_last_point
        path_key = (points_to_retrace, last_point)
        _, next_to_last_point = memo[path_key]
        points_to_retrace = tuple(sorted(set(path_key[0]).difference({last_point})))
        ind -= 1
        
    return optimal_path, optimal_cost

In [16]:
def DP_TSP(distances_array, condition_arr):
    n = len(distances_array)
    all_points_set = set(range(n))
  
    # memo keys: tuple(sorted_points_in_path, last_point_in_path)
    # memo values: tuple(cost_thus_far, next_to_last_point_in_path)
    memo = {(tuple([i]), i): tuple([0, None]) for i in range(n)}
    queue = [(tuple([0]), 0)]
  
    while queue:
        prev_visited, prev_last_point = queue.pop(0)
        prev_dist, _ = memo[(prev_visited, prev_last_point)]
        to_visit = all_points_set.difference(set(prev_visited))
        
        for new_last_point in to_visit:
            new_visited = tuple(sorted(list(prev_visited) + [new_last_point]))
            new_dist = (prev_dist + distances_array[prev_last_point][new_last_point])
            
            if len(new_visited) > (2 * len(condition_arr) + 1):
                continue
                
            key = (new_visited, new_last_point)
        
            if key not in memo:
                memo[key] = (new_dist, prev_last_point)
                queue += [key]
            else:
                # Update Cost with minimum Cost
                if new_dist < memo[key][0]:
                    memo[key] = (new_dist, prev_last_point)
          
    optimal_path, optimal_cost = retrace_optimal_path(distances_array, memo, n, condition_arr)
    return optimal_path, optimal_cost

In [17]:
point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]

print(point_arr)

start = time.process_time()
DP_TSP(adj_mat, point_arr)
print(f"Time Taken (Held-Karp): {time.process_time() - start}")

[2, 4]
((0,), 0) (0, None)
((1,), 1) (0, None)
((2,), 2) (0, None)
((3,), 3) (0, None)
((4,), 4) (0, None)
((5,), 5) (0, None)
((0, 1), 1) (135.532, 0)
((0, 2), 2) (inf, 0)
((0, 3), 3) (251.336, 0)
((0, 4), 4) (inf, 0)
((0, 5), 5) (334.646, 0)
((0, 1, 2), 2) (362.459, 1)
((0, 1, 3), 3) (342.174, 1)
((0, 1, 4), 4) (494.75300000000004, 1)
((0, 1, 5), 5) (522.398, 1)
((0, 1, 2), 1) (inf, 2)
((0, 2, 3), 3) (inf, 2)
((0, 2, 4), 4) (inf, 2)
((0, 2, 5), 5) (inf, 2)
((0, 1, 3), 1) (457.978, 3)
((0, 2, 3), 2) (408.48900000000003, 3)
((0, 3, 4), 4) (419.494, 3)
((0, 3, 5), 5) (497.36400000000003, 3)
((0, 1, 4), 1) (inf, 4)
((0, 2, 4), 2) (inf, 4)
((0, 3, 4), 3) (inf, 4)
((0, 4, 5), 5) (inf, 4)
((0, 1, 5), 1) (721.512, 5)
((0, 2, 5), 2) (499.48800000000006, 5)
((0, 3, 5), 3) (580.674, 5)
((0, 4, 5), 4) (464.203, 5)
((0, 1, 2, 3), 3) (519.612, 2)
((0, 1, 2, 4), 4) (inf, 2)
((0, 1, 2, 5), 5) (527.301, 2)
((0, 1, 2, 3), 2) (499.327, 3)
((0, 1, 3, 4), 4) (510.332, 3)
((0, 1, 3, 5), 5) (588.202, 3)
((